## Data Preparation

Step 1. Load the data.

In [23]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import kagglehub


In [3]:
# Define path to Kagglehub dataset

path = kagglehub.dataset_download("kartik2112/fraud-detection")

100%|██████████| 202M/202M [00:05<00:00, 39.3MB/s] 

Extracting files...


In [8]:
fraud_test_csv = r"C:\Users\Jean\.cache\kagglehub\datasets\kartik2112\fraud-detection\versions\1\fraudTest.csv"
fraud_train_csv = r"C:\Users\Jean\.cache\kagglehub\datasets\kartik2112\fraud-detection\versions\1\fraudTrain.csv"

In [9]:
fraud_test_df = pd.read_csv(fraud_test_csv)
fraud_train_df = pd.read_csv(fraud_train_csv)

In [20]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
fraud_test_df = pd.read_csv(fraud_test_csv)
fraud_train_df = pd.read_csv(fraud_test_csv)


In [21]:
# Check the fraud_test_df information

fraud_test_df.info()
fraud_test_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             555719 non-null  int64  
 1   trans_date_trans_time  555719 non-null  object 
 2   cc_num                 555719 non-null  int64  
 3   merchant               555719 non-null  object 
 4   category               555719 non-null  object 
 5   amt                    555719 non-null  float64
 6   first                  555719 non-null  object 
 7   last                   555719 non-null  object 
 8   gender                 555719 non-null  object 
 9   street                 555719 non-null  object 
 10  city                   555719 non-null  object 
 11  state                  555719 non-null  object 
 12  zip                    555719 non-null  int64  
 13  lat                    555719 non-null  float64
 14  long                   555719 non-nu

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,5.557190e+05,5.557190e+05,555719.000000,555719.000000,555719.000000
mean,277859.000000,4.178387e+17,69.392810,48842.628015,38.543253,-90.231325,8.822189e+04,1.380679e+09,38.542798,-90.231380,0.003860
std,160422.401459,1.309837e+18,156.745941,26855.283328,5.061336,13.721780,3.003909e+05,5.201104e+06,5.095829,13.733071,0.062008
min,0.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.371817e+09,19.027422,-166.671575,0.000000
25%,138929.500000,1.800429e+14,9.630000,26292.000000,34.668900,-96.798000,7.410000e+02,1.376029e+09,34.755302,-96.905129,0.000000
50%,277859.000000,3.521417e+15,47.290000,48174.000000,39.371600,-87.476900,2.408000e+03,1.380762e+09,39.376593,-87.445204,0.000000
75%,416788.500000,4.635331e+15,83.010000,72011.000000,41.894800,-80.175200,1.968500e+04,1.385867e+09,41.954163,-80.264637,0.000000
max,555718.000000,4.992346e+18,22768.110000,99921.000000,65.689900,-67.950300,2.906700e+06,1.388534e+09,66.679297,-66.952026,1.000000


In [22]:
# Check the fraud_train_df information

fraud_train_df.info()
fraud_train_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             555719 non-null  int64  
 1   trans_date_trans_time  555719 non-null  object 
 2   cc_num                 555719 non-null  int64  
 3   merchant               555719 non-null  object 
 4   category               555719 non-null  object 
 5   amt                    555719 non-null  float64
 6   first                  555719 non-null  object 
 7   last                   555719 non-null  object 
 8   gender                 555719 non-null  object 
 9   street                 555719 non-null  object 
 10  city                   555719 non-null  object 
 11  state                  555719 non-null  object 
 12  zip                    555719 non-null  int64  
 13  lat                    555719 non-null  float64
 14  long                   555719 non-nu

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,5.557190e+05,5.557190e+05,555719.000000,555719.000000,555719.000000
mean,277859.000000,4.178387e+17,69.392810,48842.628015,38.543253,-90.231325,8.822189e+04,1.380679e+09,38.542798,-90.231380,0.003860
std,160422.401459,1.309837e+18,156.745941,26855.283328,5.061336,13.721780,3.003909e+05,5.201104e+06,5.095829,13.733071,0.062008
min,0.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.371817e+09,19.027422,-166.671575,0.000000
25%,138929.500000,1.800429e+14,9.630000,26292.000000,34.668900,-96.798000,7.410000e+02,1.376029e+09,34.755302,-96.905129,0.000000
50%,277859.000000,3.521417e+15,47.290000,48174.000000,39.371600,-87.476900,2.408000e+03,1.380762e+09,39.376593,-87.445204,0.000000
75%,416788.500000,4.635331e+15,83.010000,72011.000000,41.894800,-80.175200,1.968500e+04,1.385867e+09,41.954163,-80.264637,0.000000
max,555718.000000,4.992346e+18,22768.110000,99921.000000,65.689900,-67.950300,2.906700e+06,1.388534e+09,66.679297,-66.952026,1.000000


Step 2. Create the labels set (y) from the “is_fraud” column, and then create the features (X) DataFrame from the remaining columns. Note: the data is already split into separate train and test datasets.

In [12]:
# Separate the data into labels and features

# Separate the y variable, the labels
y_train = fraud_train_df['is_fraud']
y_test = fraud_test_df['is_fraud']

# Separate the X variable, the features
X_train = fraud_train_df.drop(columns='is_fraud')
X_test = fraud_test_df.drop(columns='is_fraud')

In [13]:
# Review the y_train variable Series
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: is_fraud, dtype: int64

In [14]:
# Review the y_test variable Series
y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: is_fraud, dtype: int64

In [24]:
# For X_train Identify the columns that need to be preprocessed
categorical_cols = X_train.select_dtypes(include=['object']).columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns


In [26]:
# Pre-process the X_train data.
# Create a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Scale numerical columns
        ('cat', OneHotEncoder(), categorical_cols)   # Encode categorical columns
    ])

# Fit and transform the data
X_train = preprocessor.fit_transform(X_train)

In [27]:
# For X_test Identify the columns that need to be preprocessed
categorical_cols = X_test.select_dtypes(include=['object']).columns
numerical_cols = X_test.select_dtypes(include=['int64', 'float64']).columns


In [28]:
# Pre-process the X_test data.
# Create a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Scale numerical columns
        ('cat', OneHotEncoder(), categorical_cols)   # Encode categorical columns
    ])

# Fit and transform the data
X_test = preprocessor.fit_transform(X_test)

In [30]:
# Review the X variable DataFrame
X_train

<555719x1105221 sparse matrix of type '<class 'numpy.float64'>'
	with 12225817 stored elements in Compressed Sparse Row format>

In [31]:
# Review the X variable DataFrame
X_test

<555719x1105221 sparse matrix of type '<class 'numpy.float64'>'
	with 12225817 stored elements in Compressed Sparse Row format>

## Create a Logistic Regression Model with the Original Data

Step 1: Fit a logistic regression model by using the training data (X_train and y_train).

In [32]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

Step 2: Save the predictions on the testing data labels by using the testing feature data (X_test) and the fitted model.

In [33]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

predictions_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

predictions_df.head()

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


Step 3: Evaluate the model’s performance by doing the following:
* Generate a confusion matrix.

* Print the classification report.

In [34]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

print("Confusion Matrix")
cm_df

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,553499,75
Actual 1,1769,376


In [35]:
# Print the classification report for the model
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.83      0.18      0.29      2145

    accuracy                           1.00    555719
   macro avg       0.92      0.59      0.64    555719
weighted avg       1.00      1.00      1.00    555719

